In [ ]:
import numpy as np
import pickle

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
from compas.datastructures import Network
import os

import matplotlib.pyplot as plt

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

In [ ]:
def compute_perpendicular_forces(network, force_balloon, force_friction, fixed, top_ring):
    """Compute forces using the node Laplacian as a surface normal approximation."""
    forces = []
    top_ring_center = np.mean(np.array(network.nodes_attributes('xyz', keys = top_ring)), axis=0)
    for v in network.nodes():
        if v in fixed:
            forces.append(np.zeros(3))
            continue
        if network.degree(v) == 3:
            neighbors = network.neighbors(v)
            vertical_neighbor = min(neighbors, key=lambda n: network.node_attribute(n, 'z'))
            direction = np.array(network.node_attributes(v, 'xyz')) - np.array(network.node_attributes(vertical_neighbor, 'xyz'))
            direction_f = direction / np.linalg.norm(direction)
            horizontal_neighbors = [n for n in neighbors if n != vertical_neighbor]
            neigbors_vector = np.array(network.node_attributes(horizontal_neighbors[1], 'xyz')) - np.array(network.node_attributes(horizontal_neighbors[0], 'xyz'))
            neigbors_vector /= np.linalg.norm(neigbors_vector)
            direction_b = np.cross(direction, neigbors_vector)

            # Compute the vector from vertex v to the top ring center
            vertex_coords = np.array(network.node_attributes(v, 'xyz'))
            to_top_ring_center = top_ring_center - vertex_coords
            
            # Check if direction_b is pointing towards or away from top_ring_center
            if np.dot(direction_b, to_top_ring_center) < 0:
                direction_b = -direction_b  # Flip direction if it's pointing away

            # forces.append(direction_b * force_friction) # 
            forces.append(direction_f * force_friction + direction_b * force_balloon) # + 
            continue
        laplacian = network.node_laplacian(v)
        normal = laplacian / np.linalg.norm(laplacian)  # Normalize
        
        forces.append(normal * force_balloon)  # Apply force magnitude
    return np.array(forces)

def update_shape_nlf(vertices_0, loads_0, q, edges, force_balloon, force_friction, fixed, top_ring, tol = 1e-3, max_iter = 20):
    """Update the vertices of the network based on the force densities.
    parameters:
    q: list of force densities
    edges: list of edges. If not provided, the function will assume that a new q is provided for all edges in the same order as the previous q.
    """
    from compas_fd.solvers import fd_constrained_numpy
    loads = np.copy(loads_0)
    vertices = np.copy(vertices_0)
    for i in range(max_iter):
    # while True:
        result = fd_constrained_numpy(
            vertices=vertices,
            fixed= fixed,
            edges= edges,
            forcedensities=q,
            loads=loads,
            constraints= [],
        )
        netC = Network.from_nodes_and_edges(vertices, edges)
        loads_new = compute_perpendicular_forces(netC, force_balloon, force_friction, fixed, top_ring)
        if np.linalg.norm(loads - loads_new) < tol:
            break
        loads = loads_new

    vertices = np.array(result.vertices).reshape(-1, 3)
    f = np.array(result.forces).reshape(-1)
    l1 = np.array(result.lengths).reshape(-1)
    return vertices, f, l1

def cylinder_mesh(D, H, N, M, q_circumferential, q_vertical, dq):
    """Generate vertices and edges of a cylinder with adjustable vertical segments."""
    R = D / 2  # Radius
    theta = np.linspace(0, 2 * np.pi, N, endpoint=False)
    z = np.linspace(0, H, M + 1)  # M vertical segments → M+1 height levels

    # Generate vertices
    vertices = np.array([[R * np.cos(t), R * np.sin(t), h] for h in z for t in theta])

    # Define edges
    edges = []
    paths = []
    q = []

    # Circumferential edges at each level
    for j in range(M + 1):
        offset = j * N
        new_path = [(offset + i, offset + (i + 1) % N) for i in range(N)]
        edges += new_path
        paths.append(new_path)
        q += list(np.ones(N) * q_circumferential/N)
    
    # Vertical edges between stacked layers
    for j in range(M):
        offset1, offset2 = j * N, (j + 1) * N
        new_path = [(offset1 + i, offset2 + i) for i in range(N)]
        edges += new_path
        paths.append(new_path)
        q += list(np.ones(N) * q_vertical/M + np.sin(theta) * dq)

    return vertices, edges, paths, q

Define structure

In [168]:
model_name  = 'Showcase_structure_0'
D, H, N, M = 1.0, 2.0, 16, 10
q_circumferential, q_vertical, dq = 16, 10, 0.1
force_balloon, force_friction = -.1, 2
vertices_0, edges, paths, q = cylinder_mesh(D, H, N, M,q_circumferential, q_vertical, dq)
directions  = [1]*len(edges)
fixed      = np.argwhere(vertices_0[:,2] == 0).flatten()
fixed      = fixed.tolist()
top_ring = np.argwhere(vertices_0[:,2] == H).flatten()
top_ring = top_ring.tolist()

netC = Network.from_nodes_and_edges(vertices_0, edges)
loads_0 = compute_perpendicular_forces(netC, force_balloon, force_friction, fixed, top_ring)
net = Network_custom.from_fd(vertices_0, edges, q, fixed, paths = paths, dir = directions, loads=loads_0)

In [ ]:
vertices2, f2, l12 = update_shape_nlf(vertices_0, loads_0, q, edges, force_balloon, force_friction, fixed, top_ring, tol = 1e-3, max_iter = 20)

KeyboardInterrupt: 

In [160]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for edge in edges:
    ax.plot(*zip(*net.vertices[edge,:]), color='b')
for v, f in zip(net.vertices, forces):
    ax.plot([v[0], v[0] + f[0]], [v[1], v[1] + f[1]], [v[2], v[2] + f[2]], 'r-', lw=2)
ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.set_zlim(0, 14)
plt.show()

In [134]:
netC2  = Network.from_nodes_and_edges(net.vertices, edges)
forces2 = compute_perpendicular_forces(netC2, force_balloon, force_friction, fixed, top_ring)
net2 = Network_custom.from_fd(net.vertices, edges, q, fixed, paths = paths, dir = directions, loads=forces2)

In [136]:
%matplotlib qt
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for edge in edges:
    ax.plot(*zip(*net2.vertices[edge,:]), color='b')
for v, f in zip(net2.vertices, forces2):
    ax.plot([v[0], v[0] + f[0]], [v[1], v[1] + f[1]], [v[2], v[2] + f[2]], 'r-', lw=2)
ax.set_xlim(-7, 7)
ax.set_ylim(-7, 7)
ax.set_zlim(0, 14)
plt.show()

In [164]:
net.net_plot(elables=True)